### First-Mile Problem

Consider:
- 50 requests to transit stations
- 2 dummy nodes to depart and return
- time in minutes relative to the vehicle strating time
- Assume every vehicle starts at same time and works for 

In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

### 1. Data

In [2]:
# Load
sd = pd.read_csv('service_duration.csv')
df_97 = pd.read_csv('reg2_97.csv')
df_95 = pd.read_csv('reg2_95.csv')
df_123 = pd.read_csv('reg2_123.csv')
travel_time = pd.read_csv('TravelTime_dp.csv')

df = pd.concat([df_95, df_97, df_123], axis=0)
df['trip_id'] = [i for i in range(1,len(df)+1)]

# set of nodes
P = [i for i in df.trip_id]
D = [i+len(df) for i in df.trip_id]
dummy = [0, 2*len(df)+1]
N = P + D + dummy

# demand of each trip
q_p = {i: df.loc[df['trip_id'] == i, 'k_i1'].iloc[0] for i in P }
q_d = {i: df.loc[df['trip_id'] == i-len(df), 'k_i2'].iloc[0] for i in D }
q_dd = {i: 0 for i in dummy}   # is it correct?
q = {**q_p, **q_d, **q_dd} 

# early serving time
e_p = {i: df.loc[df['trip_id'] == i, 'e_i1'].iloc[0] for i in P }
e_d = {i: df.loc[df['trip_id'] == i-len(df), 'e_i2'].iloc[0] for i in D }
e_dd = {i: 0 for i in dummy}   # the earliest possible from the vehicle start dispatch time
ea = {**e_p, **e_d, **e_dd} 

# latest serving time
M = 1e6
l_p = {i: df.loc[df['trip_id'] == i, 'l_i1'].iloc[0] for i in P }
l_d = {i: df.loc[df['trip_id'] == i-len(df), 'l_i2'].iloc[0] for i in D }
l_dd = {i: M for i in dummy}   # unrestricted value
l = {**l_p, **l_d, **l_dd} 

# Node-Trip ID
node_trip_p = {i: df.loc[df['trip_id'] == i, 'reg1'].iloc[0] for i in P }
node_trip_d = {i: df.loc[df['trip_id'] == i-len(df), 'reg2'].iloc[0] for i in D }
node_trip_dd = {i: i for i in dummy}
node_trip = {**node_trip_p, **node_trip_d, **node_trip_dd}

# travel time
A = [(i, j) for i in N for j in N if i != j]  # complete graph
tij = { (i,j): travel_time.iat[node_trip[i],node_trip[j]] for i,j in A}
tij[(2*len(df)+1,0)] = 0
tij[(0,2*len(df)+1)]= 0


# maximum service time
r = 0.5
Lmax = {i: (1+r)*tij[(i,len(df)+i)] for i in P }

# service duration
# assume fix service duretion (loads and unload passenger)
s_pd = {i: 3 for i in P+D}
s_dd = {i: 0 for i in dummy}
s = {**s_pd, **s_dd}


# set of vehicles
m = 5
M = [i for i in range(1,m+1)]
Tm = {i: np.random.randint(100,400) for i in M}  # on average the ride-sharing only work 4-6 hours/day
Qmax = 4 # assume every vehicles is medium passenger cars (max 5 seats)

# weights
a1 = 0.6  
a2 = 0.2
a3 = 0.2

In [3]:
df

,e_i1,l_i1,k_i1,reg1,e_i2,l_i2,k_i2,reg2,trip_id
0,40,57,2,126,60,77,-2,95,1
1,24,47,1,98,30,53,-1,95,2
2,15,50,1,79,33,68,-1,95,3
0,30,45,1,127,44,64,-1,97,4
1,16,45,2,33,36,70,-2,97,5
0,12,41,2,77,17,55,-2,123,6
1,38,57,3,94,45,73,-3,123,7


In [4]:
2*len(df)+1

15

### 2. Problem Formulation

In [5]:
model = gp.Model('VRPTW-FMP')

Academic license - for non-commercial use only - expires 2022-01-05
Using license file /Users/denissapurba/gurobi.lic


In [6]:
# Decision Variables
x = model.addVars(M, A, lb=0, vtype=GRB.BINARY, name="x")
Q = model.addVars(M, N, lb=0, ub= Qmax, vtype=GRB.CONTINUOUS, name="Q")
B = model.addVars(M, N, lb=0, vtype=GRB.CONTINUOUS, name="B")
L = model.addVars(M, P, lb=0, vtype=GRB.CONTINUOUS, name="L")

model.update()


In [7]:
# Constraints
model.addConstrs(gp.quicksum(x[h, i, j] for j in P+D if j != i for h in M) == 1 for i in P)

model.addConstrs(gp.quicksum(x[h, j, i] for j in N if j != i) - gp.quicksum(x[h, i, k] for k in N if k != i) == 0 for i in P+D for h in M)          

model.addConstrs(gp.quicksum(x[h, i, j] for j in N if j != i) - gp.quicksum(x[h, k, len(df)+i] for k in N if k != len(df)+i) == 0 for i in P for h in M)          

model.addConstrs(gp.quicksum(x[h, j, 2*len(df)+1] for j in N if j != 2*len(df)+1) == 1 for h in M)          

model.addConstrs(gp.quicksum(x[h, 0, k] for k in N if k != 0) == 1 for h in M)          

model.addConstrs(B[h,i] >= ea[i] for i in N for h in M)

model.addConstrs(B[h,i] <= l[i] for i in N for h in M)

model.addConstrs(B[h,2*len(df)+1] - B[h,0] <= Tm[h] for h in M)

model.addConstrs(B[h,j] >= (B[h,i] + s[i] + tij[(i,j)])*x[h,i,j] for i in N for j in N if j!=i for h in M)

model.addConstrs(L[h,i] == B[h,(len(df)+i)] - (B[h,i] + s[i]) for i in P for h in M)

model.addConstrs(L[h,i] >= tij[(i,(len(df)+i))] for i in P for h in M)

model.addConstrs(L[h,i] <= Lmax[i] for i in P for h in M)

model.addConstrs(Q[h,j] >= (Q[h,i] + q[j])*x[h,i,j] for i in N for j in N if j!=i for h in M)

model.addConstrs(Q[h,i] >= q[i] for i in N for h in M)

model.addConstrs(Q[h,i] <= Qmax + q[i] for i in N for h in M)

model.update()

In [8]:
# Objective

obj1 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="obj1")  # auxiliary variable to represent objective
obj2 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="obj2")  # auxiliary variable to represent objective
obj3 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="obj3")  # auxiliary variable to represent objective

# Objective 1
model.addConstr(gp.quicksum(tij[(i,j)]*x[h,i,j] for i,j in A for h in M) <= obj1)

# Objective 2
model.addConstr(gp.quicksum(L[h,i] - tij[(i,(len(df)+i))] for i in P for h in M) <= obj2)

# Objective 3
model.addConstr(gp.quicksum((B[h,i] - ea[i])*q[i] for i in P for h in M) <= obj3)

model.setObjective((a1*obj1 + a2*obj2 + a3*obj3), GRB.MINIMIZE)

model.update()


In [9]:
model.write("VRPTW-FMP.lp")

In [10]:
#Set Parameter as Nonconvex Optimization Problem
model.Params.NonConvex = 2
model.Params.MIPFocus = 3
#model.Params.NumericFocus = 3
#model.Params.IntFeasTol = 1e-08

Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter MIPFocus to 3
   Prev: 0  Min: 0  Max: 3  Default: 0


In [11]:
model.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 555 rows, 1398 columns and 5403 nonzeros
Model fingerprint: 0x5cb92bc7
Model has 2400 quadratic constraints
Variable types: 198 continuous, 1200 integer (1200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 5e+01]
  Objective range  [2e-01, 6e-01]
  Bounds range     [1e+00, 4e+00]
  RHS range        [1e+00, 1e+06]
Presolve added 657 rows and 0 columns
Presolve removed 0 rows and 478 columns
Presolve time: 0.10s
Presolved: 1792 rows, 1210 columns, 11050 nonzeros
Variable types: 475 continuous, 735 integer (735 binary)
Presolved: 1493 rows, 873 columns, 10057 nonzeros


Root relaxation: objective 1.467000e+02, 321 iterations, 0.08 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    

In [12]:
for m in M:
    for u,v in A:
        if x[m,u,v].X==1:
            print(m,u,v)

1 5 12
1 12 15
1 0 5
2 4 11
2 11 15
2 0 4
3 7 14
3 14 15
3 0 7
4 2 10
4 3 2
4 9 15
4 10 9
4 0 3
5 1 8
5 6 13
5 8 15
5 13 1
5 0 6


In [18]:
B[5,6]

<gurobi.Var B[5,6] (value 12.999999999999996)>

In [19]:
df

,e_i1,l_i1,k_i1,reg1,e_i2,l_i2,k_i2,reg2,trip_id
0,40,57,2,126,60,77,-2,95,1
1,24,47,1,98,30,53,-1,95,2
2,15,50,1,79,33,68,-1,95,3
0,30,45,1,127,44,64,-1,97,4
1,16,45,2,33,36,70,-2,97,5
0,12,41,2,77,17,55,-2,123,6
1,38,57,3,94,45,73,-3,123,7
